In [9]:
!pip install pyspark

In [24]:
import os
import sys
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql import Row
from pyspark.sql.functions import desc


# Настройка переменных окружения для PySpark
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
# Добавление пакета для парсинга XML
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.17.0 pyspark-shell'


In [11]:
# Создание SparkSession
spark = SparkSession.builder.appName("2010-2020 top programming languages").getOrCreate()

In [12]:
# Загрузка файлов с данными
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/posts_sample.xml
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/programming-languages.csv

--2025-04-21 15:18:34--  https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/posts_sample.xml
Resolving git.ai.ssau.ru (git.ai.ssau.ru)... 91.222.131.161
Connecting to git.ai.ssau.ru (git.ai.ssau.ru)|91.222.131.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74162295 (71M) [text/plain]
Saving to: ‘posts_sample.xml.1’

posts_sample.xml.1  100%[===================>]  70.73M   532KB/s    in 2m 14s  

2025-04-21 15:20:49 (540 KB/s) - ‘posts_sample.xml.1’ saved [74162295/74162295]

--2025-04-21 15:20:49--  https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/programming-languages.csv
Resolving git.ai.ssau.ru (git.ai.ssau.ru)... 91.222.131.161
Connecting to git.ai.ssau.ru (git.ai.ssau.ru)|91.222.131.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40269 (39K) [text/plain]
Saving to: ‘programming-languages.csv.1’

programming-languag 100%[===================>]  39.33K   129KB/s    in 0.3s    

2025-04-21 15:20:50 (129 KB/s)

**Загрузка и первичный анализ данных из  posts_sample.xml**

In [16]:
# Импорт данных из XML-файла в объект DataFrame
post_records = spark.read.format('xml').option('rowTag', 'row').option("timestampFormat", 'yyyy/MM/dd HH:mm:ss').load('posts_sample.xml')

# Анализ структуры загруженных данных
print("Структура таблицы с записями:")
post_records.printSchema()
print("\nПримеры записей (первые 5 строк):")
post_records.limit(5).show()
print("\nСводная статистика по числовым полям:")
post_records.summary().show()
print(f"Всего записей: {post_records.count()}")

Структура таблицы с записями:
root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)


Примеры записей (первые 5 строк):
+-----------

**Фильтрация записей по временному периоду**

In [18]:
# Задание временного интервал
time_range = ("2010-01-01", "2020-12-31")

# Отбор записей, попадающих в указанный период
filtered_posts = post_records.where(F.col("_CreationDate").between(time_range[0], time_range[1]))

# Визуализация результата фильтрации
print("Отфильтрованные записи (первые 10):")
filtered_posts.limit(10).show()

Отфильтрованные записи (первые 10):
+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+-----+------+----------+
|_AcceptedAnswerId|_AnswerCount|               _Body|_ClosedDate|_CommentCount| _CommunityOwnedDate|       _CreationDate|_FavoriteCount|    _Id|   _LastActivityDate|       _LastEditDate|_LastEditorDisplayName|_LastEditorUserId|_OwnerDisplayName|_OwnerUserId|_ParentId|_PostTypeId|_Score|_Tags|_Title|_ViewCount|
+-----------------+------------+--------------------+-----------+-------------+--------------------+--------------------+--------------+-------+--------------------+--------------------+----------------------+-----------------+-----------------+------------+---------+-----------+------+-----+------+----------+
|             NULL|        N

**Загрузка и первичный анализ данных из  programming-languages.csv**

In [20]:
# Импорт CSV-файла с информацией о языках программирования
prog_langs_df = spark.read.format('com.databricks.spark.csv').option('header', 'true').option("inferSchema", "true").load('programming-languages.csv').na.drop()

# Анализ структуры загруженных данных
print("Структура таблицы с языками программирования:")
prog_langs_df.printSchema()
print("\nПримеры данных (первые 5 записей):")
prog_langs_df.limit(5).show()
print("\nОсновные статистические показатели:")
prog_langs_df.summary().show()
print(f"Найдено записей: {prog_langs_df.count()}")

Структура таблицы с языками программирования:
root
 |-- name: string (nullable = true)
 |-- wikipedia_url: string (nullable = true)


Примеры данных (первые 5 записей):
+----------+--------------------+
|      name|       wikipedia_url|
+----------+--------------------+
|   A# .NET|https://en.wikipe...|
|A# (Axiom)|https://en.wikipe...|
|A-0 System|https://en.wikipe...|
|        A+|https://en.wikipe...|
|       A++|https://en.wikipe...|
+----------+--------------------+


Основные статистические показатели:
+-------+--------+--------------------+
|summary|    name|       wikipedia_url|
+-------+--------+--------------------+
|  count|     699|                 699|
|   mean|    NULL|                NULL|
| stddev|    NULL|                NULL|
|    min|@Formula|https://en.wikipe...|
|    25%|    NULL|                NULL|
|    50%|    NULL|                NULL|
|    75%|    NULL|                NULL|
|    max|xHarbour|https://en.wikipe...|
+-------+--------+--------------------+

Найден

**Определение языков программирования в тегах постов**

In [22]:
# Получаем список названий языков программирования
prog_lang_list = [row.name for row in prog_langs_df.select("name").collect()]

# Создаем RDD с информацией о наличии языков в тегах
posts_with_langs = filtered_posts.rdd.map(lambda post: (post._CreationDate, next((lang for lang in prog_lang_list if f"<{lang.lower()}>" in str(post._Tags).lower()), 'No')))

# Преобразуем обратно в DataFrame с указанием схемы
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

result_schema = StructType([StructField("creation_date", TimestampType()), StructField("language", StringType())])

posts_langs_df = spark.createDataFrame(posts_with_langs, schema=result_schema)

**Анализ популярности языков и создание отчета**

In [30]:
# Преобразование DataFrame в RDD с определением языков в тегах
posts_rdd = filtered_posts.rdd.map(lambda post: (post._CreationDate, next((lang for lang in prog_lang_list if f"<{lang.lower()}>" in str(post._Tags).lower()), 'No'))).filter(lambda x: x[1] != 'No')

# Группировка и подсчет по году и языку
grouped_stats = posts_rdd.keyBy(lambda row: (row[0].year, row[1])).aggregateByKey(0, lambda acc, _: acc + 1, lambda acc1, acc2: acc1 + acc2).sortBy(lambda x: x[1], ascending=False).collect()

# Формирование списка годов в обратном порядке
years = list(range(2010, 2021))[::-1]

# Сбор топ-10 языков для каждого года
top_langs = []
for year in years:
    yearly_data = [row for row in grouped_stats if row[0][0] == year]
    top_langs.extend([(year, lang, count) for ((y, lang), count) in yearly_data[:10]])

# Создание DataFrame через Row-объекты
from pyspark.sql import Row
result_row = Row('Year', 'Language', 'Count')
result_df = spark.createDataFrame([result_row(*data) for data in top_langs])

# Вывод и сохранение результатов
print("Топ-10 языков программирования по годам (2010-2020):")
result_df.orderBy("Year", "Count", ascending=[False, False]).show()

result_df.write.mode("overwrite").parquet("top_10_languages_by_year.parquet")

Топ-10 языков программирования по годам (2010-2020):
+----+----------+-----+
|Year|  Language|Count|
+----+----------+-----+
|2019|    Python|  162|
|2019|JavaScript|  131|
|2019|      Java|   95|
|2019|       PHP|   59|
|2019|         R|   36|
|2019|         C|   14|
|2019|      Dart|    9|
|2019|    MATLAB|    9|
|2019|        Go|    9|
|2019|      Bash|    8|
|2018|    Python|  214|
|2018|JavaScript|  196|
|2018|      Java|  145|
|2018|       PHP|   99|
|2018|         R|   63|
|2018|         C|   24|
|2018|     Scala|   22|
|2018|TypeScript|   21|
|2018|PowerShell|   13|
|2018|      Bash|   12|
+----+----------+-----+
only showing top 20 rows

